In [1]:
from src.all_in_one import *
from src.basic_variables import *

2024-10-24 10:38:15,724 - INFO - Processed 711 records so far.
2024-10-24 10:38:16,659 - INFO - Processed 328 records so far.


## Initiate Intake Dataset

In [2]:
int_file = path_manager.get_data_file(Category.PS, PSFile.PSNT)
intake = get_df(int_file.path, int_file.sheet)
intake_dataset = Dataset(config_file, int_file.sheet)
# filtered_intake = filter_dataframe_on_date(intake, intake_dataset.dvars, START, END, FilterType.IN)

In [4]:
intake.head()

## Filter New Intake

In [16]:
new_nt_s1 = filter_dataframe_on_date(intake, "nt_s1", START, END)
new_intake = filter_dataframe_on_date(intake, "nt_s2", START, END)
new_nt_s3 = filter_dataframe_on_date(intake, "nt_s3", START, END)
new_reintake = filter_dataframe_on_date(intake, "nt_re", START, END)

In [38]:
# new_intake

In [39]:
# new_reintake

## Intake Number of Sessions

In [20]:
nt_s1 = new_nt_s1.shape[0]
nt_s2 = new_intake.shape[0]
nt_s3 = new_nt_s3.shape[0]
nt_re = new_reintake.shape[0]

In [40]:
# nt_s1 + nt_s2 + nt_s3 + nt_re

## New Intake Sex and Age Groups

In [41]:
f_new_intake = pd.merge(
    new_intake, rebuilt_scr[["rid", "sex", "age"]], on="rid", how="left"
)
# f_new_intake[['rid', 'fcid', 'sex', 'age']]

In [42]:
# f_new_intake['sex'].value_counts(dropna=False, normalize=True)

In [43]:
# categorize_and_count_ages(f_new_intake['age'])

## Beneficiaries Victimhood

In [35]:
# intake_dataset.bovars

In [36]:
count_df = new_intake[intake_dataset.bovars].apply(lambda x: x.value_counts()).fillna(0)
count_df = count_df.T.reset_index().rename(
    columns={"index": "victimhood"}
)  # Transposing
count_df["No"] = count_df["No"].astype("Int64")
count_df["Yes"] = count_df["Yes"].astype("Int64")


In [44]:
# count_df